# Allele-Specific Expression

RNA sequencing can distinguish transcripts expressed from different copies of genes on homologous chromosomes when single-nucleotide polymorphisms (perhaps silent) distinguish the two alleles. Linkage between these distinctive SNPs and _cis_-regulatory sequences can provide information on regulatory variation within a shared cellular context.

## Null Hypothesis Testing

The null hypothesis in allele-specific expression analysis is that the alleles are expressed equally and so each read is equally likely to be derived from each allele.

Here, we'll take two approaches to get a _p_ value for the null hypothesis of equal expression in situations where just 25% of the reads come from one allele and 75% from the other. We'll look at this with just 8 reads, with 32 reads, and then with 100 reads.

### Permutation Testing

First, we'll generate many random sets of data according to the null model and look at the distribution of allele skew in these random data. Our approach to generating random sets of reads is simple: we choose randomly between `0` and `1`, and then count how often we choose `1` by summing the results of this random choice.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
reads = np.random.choice(np.array([0, 1]), 8)
print(reads)
print(sum(reads))

**Exercise** Complete the work below to generate 10,000 random samples of 8 reads each, and tabulate how many random samples have zero, one, ..., eight reads from that sample. Print the counts, and then plot the distribution.

In [ ]:
allele_counts_8 = np.zeros(9)
for i in range(0,10000):
    n_reads = ...
    allele_counts_8[n_reads] += 1
print(allele_counts_8)
...

**Exercise** In what fraction of random samples do you see 25% or fewer `1` reads? What can you conclude from seeing this kind of skew in a sample of 8 reads?

In [ ]:
print(...)

**Exercise** Now repeat the same analysis, for 32 reads per random sample.

In [ ]:
allele_counts_32 = ...
...

**Exercise** In what fraction of 32-read random samples do you see 25% or fewer `1` reads? What can you conclude from seeing this kind of skew in a sample of 32 reads? Are there any concerns that might arise in running this analysis genome-wide?

In [ ]:
print(...)

Clearly, read number has a big impact on different outcomes. 

**Exercise** Complete the function below that runs the entire allele skew analysis using the number of reads as an input parameter, and then run it for 32 and 100 reads. 

In [ ]:
def allele_skew(num_reads):
    allele_counts = ...
    for i in range(0,10000):
        ...
    print(allele_counts)
    plt.figure()
    plt.plot(allele_counts)
    return ...
print(allele_skew(32))
print(allele_skew(100))

### Binomial Distribution

There is probably a small but non-zero odds of getting a 25% skew in a 100-read sample -- but we would need to generate a lot of random samples in order to figure out exactly how small. Instead, we could model the number of reads according to the binomial distribution. This isn't always a fair description of biological data, but it's a reasonable starting point here.

Below, we use the binomial probability distribution to plot the _expected_ distribution of counts on top of the actual random sample from our `allele_skew` function.

In [ ]:
from scipy.stats import binom
allele_skew(32)
x = np.arange(0,33)
plt.plot(binom.pmf(x, 32, 0.5) * 10000, 'o-r')

In [ ]:
sum(binom.pmf(range(0,9), 32, 0.5))

**Exercise** Compute the likelihood of getting 25 _or fewer_ `1` reads. How many random samples would you typically need to see one skewed this much?

In [ ]:
print(...)

## Maximum Likelihood Estimation

Instead of simply testing whether allele expression is even, we want to _estimate_ the relative skew in expression. To do this, we will start by making a graph where we consider all possible bias values, and then figure out the likelihood function P( 8 reads out of 32 | bias ). We'll use the `binom.pmf` function again, but now we'll consider many different values for the 3rd _p_ parameter instead of the first one.

This graph looks similar to others that we've made, but the axes are different -- we'll add x and y axis labels to emphasize this difference.

In [ ]:
bias = np.arange(0,1,0.01)
plt.plot(bias, binom.pmf(8, 32, bias))
plt.xlabel('Allele bias')
plt.ylabel('P(8 reads / 32 total)')

As we discussed before, we often want to work with log likelihood functions. In fact, `binom` has the built-in ability to give us a log probability that will be numerically stable when the actual likelihood is a very tiny number, using the `logpmf` method.

In [ ]:
plt.figure()
plt.plot(x, binom.logpmf(8, 32, x))

In this plot, the likelihood function looks pretty flat around 0.25, but we might want to adjust the y-axis to focus on the region of high likelihood -- the default puts a lot of emphasis on parts of the plot where the likelihood is very small.

In [ ]:
plt.figure()
plt.plot(x, binom.logpmf(8, 32, x))
plt.axis([0.0, 1.0, -10, 0])

We need to find the point on the x-axis where the likelihood is maximized. We can probably guess that this will happen at 0.25, but we can use algorithms from Scipy to find the best likelihood. These methods are typically expressed in terms of _minimization_, and so we'll minimize the negative log likelihood which is equivalent to finding the maximum of the likelihood.

To do this, we define a function to compute the negative log likelihood, called `negloglik`, and then use `minimize_scalar` from `scipy.optimize` to find the allele skew value that maximizes the likelihood of our data.

In [ ]:
from scipy.optimize import minimize_scalar
def negloglik(bias):
    return -binom.logpmf(8, 32, bias)
mle = minimize_scalar(negloglik, bounds=(0,1), method='bounded')
print(mle)

The width of the likelihood peak can be used to compute a confidence interval. For a 95% confidence interval, we find the range of values where the log likelihood is within 1.92 of the best log likelihood.

In [ ]:
bias_opt = mle.x
loglik_opt = binom.logpmf(8, 32, bias_opt)
print(loglik_opt, loglik_opt - 1.92)

We can do this in a brute-force sort of way -- start at the best bias value and keep increasing it until the log likelihood drops off too much:

In [ ]:
upper = bias_opt
while binom.logpmf(8, 32, upper) > loglik_opt - 1.92:
    upper += 0.001
print(upper)

Alternately, we can use algorithms in Python to solve for the point where the log likelihood crosses our threshold. That is, we want to solve for _u_ where
```
lnP(8 reads out of 32 | bias = u) = loglik_opt - 1.92
```
or
```
lnP(8 reads out of 32 | bias = u) - loglik_opt + 1.92 = 0
```

In [ ]:
from scipy.optimize import root_scalar
def confint(u):
    return binom.logpmf(8, 32, u) - loglik_opt + 1.92
upper_ci = root_scalar(confint, bracket=[bias_opt, 1.0])
print(upper_ci)

We can then use the same trick to find the lower bound of the confidence interval by looking below bias_opt -- between 0 and bias_opt -- rather than above bias_opt

In [ ]:
lower_ci = root_scalar(confint, bracket=[0.0, bias_opt])
print(lower_ci)

We can then find the overall confidence interval for our estimate, based on the shape of the likelihood function:

In [ ]:
print(lower_ci.root, upper_ci.root)

## Maximum A Posteriori Estimation

**Exercise** Below, plot the log likelihood functions for:
* 2 reads out of 8
* 8 reads out of 32
* 25 reads out of 100

In [ ]:
x = np.arange(0,1,0.01)
plt.plot(...)
...
plt.axis([0,1,-10,0])